In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

In [0]:
df = (spark
      .read
      .format('csv')
      .option("header","true")
      .option("inferSchema","true")
      .load("dbfs:/dbfs/by-day-spark-output/**/*.csv")
      )

df.createOrReplaceTempView("df")
df.printSchema()

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)



In [0]:
from pyspark.sql.functions import col
(df
 .where(col("InvoiceNo") == 536365)
 .select("InvoiceNo", "Description")
 .limit(5)
 .display()
)

InvoiceNo,Description
536365,WHITE HANGING HEART T-LIGHT HOLDER
536365,WHITE METAL LANTERN
536365,CREAM CUPID HEARTS COAT HANGER
536365,KNITTED UNION FLAG HOT WATER BOTTLE
536365,RED WOOLLY HOTTIE WHITE HEART.


In [0]:
from pyspark.sql.functions import instr

df.filter(instr(df.Description,"POSTAGE")>1).limit(5).display()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
580610,DOT,DOTCOM POSTAGE,1,2011-12-05T11:48:00Z,2196.67,null,United Kingdom
580612,DOT,DOTCOM POSTAGE,1,2011-12-05T11:58:00Z,2114.0,null,United Kingdom
580727,DOT,DOTCOM POSTAGE,1,2011-12-05T17:17:00Z,1599.26,14096.0,United Kingdom
580729,DOT,DOTCOM POSTAGE,1,2011-12-05T17:24:00Z,1172.1,null,United Kingdom
580730,DOT,DOTCOM POSTAGE,1,2011-12-05T17:28:00Z,845.23,null,United Kingdom


In [0]:
price_filter = col("UnitPrice") > 600
description_filter = instr(col("Description"), "POSTAGE") > 1

(
    df
    .where(col('StockCode').isin("DOT"))
    .where(price_filter | description_filter)
    .limit(5)
    .display()
)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
580610,DOT,DOTCOM POSTAGE,1,2011-12-05T11:48:00Z,2196.67,null,United Kingdom
580612,DOT,DOTCOM POSTAGE,1,2011-12-05T11:58:00Z,2114.0,null,United Kingdom
580727,DOT,DOTCOM POSTAGE,1,2011-12-05T17:17:00Z,1599.26,14096.0,United Kingdom
580729,DOT,DOTCOM POSTAGE,1,2011-12-05T17:24:00Z,1172.1,null,United Kingdom
580730,DOT,DOTCOM POSTAGE,1,2011-12-05T17:28:00Z,845.23,null,United Kingdom


In [0]:
spark.sql("select * \
          from df \
            where StockCode = 'DOT' \
            and (UnitPrice> 600 or instr(Description, 'POSTAGE')>1) \
          limit 5").display()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
580610,DOT,DOTCOM POSTAGE,1,2011-12-05T11:48:00Z,2196.67,null,United Kingdom
580612,DOT,DOTCOM POSTAGE,1,2011-12-05T11:58:00Z,2114.0,null,United Kingdom
580727,DOT,DOTCOM POSTAGE,1,2011-12-05T17:17:00Z,1599.26,14096.0,United Kingdom
580729,DOT,DOTCOM POSTAGE,1,2011-12-05T17:24:00Z,1172.1,null,United Kingdom
580730,DOT,DOTCOM POSTAGE,1,2011-12-05T17:28:00Z,845.23,null,United Kingdom


In [0]:
dot_code_filter = col("StockCode") == "DOT"
price_filter = col("UnitPrice") > 600
description_filter = instr(col("Description"), "POSTAGE") > 1

(
    df
    .withColumn("is_expensive", dot_code_filter & (price_filter | description_filter))
    .where(col("is_expensive"))
    .limit(5)
    .display()
)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,is_expensive
580610,DOT,DOTCOM POSTAGE,1,2011-12-05T11:48:00Z,2196.67,null,United Kingdom,true
580612,DOT,DOTCOM POSTAGE,1,2011-12-05T11:58:00Z,2114.0,null,United Kingdom,true
580727,DOT,DOTCOM POSTAGE,1,2011-12-05T17:17:00Z,1599.26,14096.0,United Kingdom,true
580729,DOT,DOTCOM POSTAGE,1,2011-12-05T17:24:00Z,1172.1,null,United Kingdom,true
580730,DOT,DOTCOM POSTAGE,1,2011-12-05T17:28:00Z,845.23,null,United Kingdom,true


In [0]:
%sql
select *, (StockCode = "DOT" and ((UnitPrice > 600) or instr(Description, "POSTAGE") > 1) ) as InExpensive
from df 
where StockCode = "DOT" 
    and ((UnitPrice > 600) or instr(Description, "POSTAGE") > 1) 
limit 5

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InExpensive
580610,DOT,DOTCOM POSTAGE,1,2011-12-05T11:48:00Z,2196.67,null,United Kingdom,true
580612,DOT,DOTCOM POSTAGE,1,2011-12-05T11:58:00Z,2114.0,null,United Kingdom,true
580727,DOT,DOTCOM POSTAGE,1,2011-12-05T17:17:00Z,1599.26,14096.0,United Kingdom,true
580729,DOT,DOTCOM POSTAGE,1,2011-12-05T17:24:00Z,1172.1,null,United Kingdom,true
580730,DOT,DOTCOM POSTAGE,1,2011-12-05T17:28:00Z,845.23,null,United Kingdom,true


In [0]:
from pyspark.sql.functions import expr

(
    df
    .withColumn("is_expensive", expr('UnitPrice > 250'))
    .limit(5)
    .display()
)

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,is_expensive
580538,23084,RABBIT NIGHT LIGHT,48,2011-12-05T08:38:00Z,1.79,14075.0,United Kingdom,false
580538,23077,DOUGHNUT LIP GLOSS,20,2011-12-05T08:38:00Z,1.25,14075.0,United Kingdom,false
580538,22906,12 MESSAGE CARDS WITH ENVELOPES,24,2011-12-05T08:38:00Z,1.65,14075.0,United Kingdom,false
580538,21914,BLUE HARMONICA IN BOX,24,2011-12-05T08:38:00Z,1.25,14075.0,United Kingdom,false
580538,22467,GUMBALL COAT RACK,6,2011-12-05T08:38:00Z,2.55,14075.0,United Kingdom,false


In [0]:
from pyspark.sql.functions import expr, pow

fabricated_quantity = pow(col("Quantity") * col("UnitPrice") , 2) + 5
(
    df
    .select("CustomerId", fabricated_quantity.alias("Quantity"))
    .limit(5)
    .display()
)

CustomerId,Quantity
14075.0,7387.2464
14075.0,630.0
14075.0,1573.1599999999996
14075.0,905.0
14075.0,239.08999999999997


In [0]:
%sql
select CustomerID, pow(Quantity * UnitPrice, 2) + 5 as NewPrice from df limit 5

CustomerID,NewPrice
14075.0,7387.2464
14075.0,630.0
14075.0,1573.1599999999996
14075.0,905.0
14075.0,239.08999999999997


In [0]:
(
    df
    .selectExpr("CustomerId", "(POWER(Quantity * UnitPrice, 2) + 5) as Quantity")
    .limit(5)
    .display()
)

CustomerId,Quantity
14075.0,7387.2464
14075.0,630.0
14075.0,1573.1599999999996
14075.0,905.0
14075.0,239.08999999999997


In [0]:
from pyspark.sql.functions import lit, round, bround

(
    df
    .select(round(lit("2.5")), bround(lit("2.5")))
    .limit(5)
    .display()
)

"round(2.5, 0)","bround(2.5, 0)"
3.0,2.0
3.0,2.0
3.0,2.0
3.0,2.0
3.0,2.0


In [0]:
%sql
select round(2.5), bround(2.5) from df limit 5;

"round(2.5, 0)","bround(2.5, 0)"
3,2
3,2
3,2
3,2
3,2


In [0]:
from pyspark.sql.functions import corr

In [0]:
df.stat.corr("Quantity", "UnitPrice")

-0.001234924544870288

In [0]:
df.select(corr("Quantity", "UnitPrice")).show()

+-------------------------+
|corr(Quantity, UnitPrice)|
+-------------------------+
|     -0.00123492454487...|
+-------------------------+



In [0]:
%sql
select corr(Quantity, UnitPrice) from df;

"corr(Quantity, UnitPrice)"
-0.001234924544870288


In [0]:
df.describe().display()

summary,InvoiceNo,StockCode,Description,Quantity,UnitPrice,CustomerID,Country
count,541909,541909,540455,541909,541909,406829,541909
mean,559965.752026781,27089.272460352007,20713.0,9.55224954743324,4.611113626089724,15287.690570239585,null
stddev,13428.417280794756,15977.952954078419,null,218.08115785022977,96.75985306117883,1713.6003033215982,null
min,536365,10002,"""ASSORTED FLOWER COLOUR """"LEIS""""""",-80995,-11062.06,12346.0,Australia
max,C581569,m,wrongly sold sets,80995,38970.0,18287.0,Unspecified


In [0]:
from pyspark.sql.functions import count, mean, stddev, min, max


df.select(mean(col('Quantity')), stddev(col('Quantity')), count(col('Quantity')), min(col('Quantity')), max(col('Quantity'))).display()

avg(Quantity),stddev(Quantity),count(Quantity),min(Quantity),max(Quantity)
9.55224954743324,218.08115785023216,541909,-80995,80995


### Window function

In [0]:
# Rank Items by Quantity per Invoice

from pyspark.sql.functions import rank
from pyspark.sql.window import Window

window_spec = Window.partitionBy("InvoiceNo").orderBy(col("Quantity").desc())

df_with_rank = df.withColumn("item_rank", rank().over(window_spec))

df_with_rank.limit(5).display()

InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,item_rank
536366,22633,HAND WARMER UNION JACK,6,2010-12-01T08:28:00Z,1.85,17850.0,United Kingdom,1
536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01T08:28:00Z,1.85,17850.0,United Kingdom,1
536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01T08:34:00Z,1.69,13047.0,United Kingdom,1
536367,22749,FELTCRAFT PRINCESS CHARLOTTE DOLL,8,2010-12-01T08:34:00Z,3.75,13047.0,United Kingdom,2
536367,22745,POPPY'S PLAYHOUSE BEDROOM,6,2010-12-01T08:34:00Z,2.1,13047.0,United Kingdom,3


In [0]:
from pyspark.sql.functions import sum, sum_distinct

(
    df
    .agg(sum("Quantity").alias("total_quantity"), sum_distinct("Quantity").alias("distinct_quantity_sum"))
    .display()
    )

total_quantity,distinct_quantity_sum
5176450,29310


In [0]:
from pyspark.sql.functions import count, countDistinct
df.agg(count("StockCode").alias('stock_code_count'), countDistinct("StockCode").alias('stock_code_distinct_count')).show()


+----------------+-------------------------+
|stock_code_count|stock_code_distinct_count|
+----------------+-------------------------+
|          541909|                     4070|
+----------------+-------------------------+



In [0]:
df.select(count("StockCode").alias('stock_code_count'), countDistinct("StockCode").alias('stock_code_distinct_count')).show()

+----------------+-------------------------+
|stock_code_count|stock_code_distinct_count|
+----------------+-------------------------+
|          541909|                     4070|
+----------------+-------------------------+



In [0]:
from pyspark.sql.functions import first, last

df.select(first("InvoiceNo"), last("InvoiceNo")).show()

+----------------+---------------+
|first(InvoiceNo)|last(InvoiceNo)|
+----------------+---------------+
|          580538|         543282|
+----------------+---------------+



In [0]:
%sql
select first(InvoiceNo), last(InvoiceNo), min(InvoiceNo), max(InvoiceNo) from df

first(InvoiceNo),last(InvoiceNo),min(InvoiceNo),max(InvoiceNo)
580538,543282,536365,C581569


In [0]:
from pyspark.sql.functions import min, max

df.select(min("InvoiceNo"), max("InvoiceNo")).show()

+--------------+--------------+
|min(InvoiceNo)|max(InvoiceNo)|
+--------------+--------------+
|        536365|       C581569|
+--------------+--------------+



In [0]:
from pyspark.sql.functions import sum, count, avg, expr

(
    df
    .select(count("Quantity").alias("count_quantity"),
            sum("Quantity").alias("sum_quantity"),
            avg("Quantity").alias("avg_quantity"),
            expr("mean(Quantity)").alias("mean_quantity"))
    .selectExpr("sum_quantity/count_quantity", "avg_quantity", "mean_quantity")
    .show()
)

+-------------------------------+----------------+----------------+
|(sum_quantity / count_quantity)|    avg_quantity|   mean_quantity|
+-------------------------------+----------------+----------------+
|               9.55224954743324|9.55224954743324|9.55224954743324|
+-------------------------------+----------------+----------------+



In [0]:
%sql
select sum_quantity/count_quantity as sum_by_count, avg_quantity, mean_quantity from 
(select mean(Quantity) as mean_quantity, avg(Quantity) as avg_quantity, sum(Quantity) as sum_quantity, count(Quantity) as count_quantity from df)

sum_by_count,avg_quantity,mean_quantity
9.55224954743324,9.55224954743324,9.55224954743324


In [0]:
from pyspark.sql.functions import collect_set, collect_list

df.select(collect_set("StockCode").alias("stock_collection_set"), collect_list("StockCode").alias("stock_collection_list")).show()

+--------------------+---------------------+
|stock_collection_set|stock_collection_list|
+--------------------+---------------------+
|[10002, 10080, 10...| [10002, 10002, 10...|
+--------------------+---------------------+



In [0]:
%sql
select collect_set(StockCode), collect_list(StockCode) from df;

com.databricks.backend.common.rpc.SparkDriverExceptions$SQLExecutionException: java.lang.Exception: Results too large
	at com.databricks.backend.daemon.driver.OutputAggregator$.maybeApplyOutputAggregation(OutputAggregator.scala:516)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation0(OutputAggregator.scala:337)
	at com.databricks.backend.daemon.driver.OutputAggregator$.withOutputAggregation(OutputAggregator.scala:101)
	at com.databricks.backend.daemon.driver.DriverLocal.executeSql(DriverLocal.scala:464)
	at com.databricks.backend.daemon.driver.JupyterDriverLocal.repl(JupyterDriverLocal.scala:1054)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$33(DriverLocal.scala:1195)
	at com.databricks.unity.EmptyHandle$.runWith(UCSHandle.scala:133)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$28(DriverLocal.scala:1186)
	at com.databricks.logging.AttributionContextTracing.$anonfun$withAttributionContext$1(AttributionContex

In [0]:
(
    df
    .groupBy("InvoiceNo", "CustomerId")
    .count()
    .limit(5)
    .show()
)

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   580657|   14696.0|   20|
|   581474|   12748.0|   24|
|   576641|   17549.0|    4|
|   575753|   17841.0|   80|
|   537252|      NULL|    1|
+---------+----------+-----+



In [0]:
%sql
select InvoiceNo, CustomerId, count(*) from df group by InvoiceNo, CustomerId limit 5;

InvoiceNo,CustomerId,count(1)
580657,14696.0,20
581474,12748.0,24
576641,17549.0,4
575753,17841.0,80
537252,null,1


In [0]:
(
    df
    .groupBy('InvoiceNo')
    .agg(count("Quantity").alias('count_quantity'),
         expr("count(Quantity)"))
    .limit(5)
    .show()
)

+---------+--------------+---------------+
|InvoiceNo|count_quantity|count(Quantity)|
+---------+--------------+---------------+
|   574966|             8|              8|
|   575091|            38|             38|
|   578057|            28|             28|
|   537252|             1|              1|
|   578459|             8|              8|
+---------+--------------+---------------+



In [0]:
(
    df
    .groupBy('InvoiceNo')
    .agg(expr("avg(Quantity)").alias('avg_quantity'), expr('stddev_pop(Quantity)').alias('std_dev_quantity'))
    .limit(5)
    .show()
)

+---------+------------------+-----------------+
|InvoiceNo|      avg_quantity| std_dev_quantity|
+---------+------------------+-----------------+
|   574966|               6.0|3.640054944640259|
|   575091|11.552631578947368|5.008925551458656|
|   578057| 4.607142857142857|8.755974636597271|
|   537252|              31.0|              0.0|
|   578459|              28.0|             26.0|
+---------+------------------+-----------------+



In [0]:
%sql
select avg(Quantity), stddev_pop(Quantity) from df limit 5;

avg(Quantity),stddev_pop(Quantity)
9.55224954743324,218.08095663447597


In [0]:
spark.sql("select avg(Quantity), stddev_pop(Quantity) from df limit 5").show()

+----------------+--------------------+
|   avg(Quantity)|stddev_pop(Quantity)|
+----------------+--------------------+
|9.55224954743324|  218.08095663447597|
+----------------+--------------------+



In [0]:
from pyspark.sql.functions import col, to_date

df_with_date = df.withColumn("date", to_date(col("InvoiceDate")))
df_with_date.createOrReplaceTempView("df_with_date")

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc

windowSpec = Window\
.partitionBy("CustomerId", "date")\
.orderBy(desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [0]:
from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [0]:
from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

In [0]:
df_with_date.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).limit(5).display()


CustomerId,date,Quantity,quantityRank,quantityDenseRank,maxPurchaseQuantity
12346.0,2011-01-18,74215,1,1,74215
12346.0,2011-01-18,-74215,2,2,74215
12347.0,2010-12-07,36,1,1,36
12347.0,2010-12-07,30,2,2,36
12347.0,2010-12-07,24,3,3,36


In [0]:
%sql

SELECT CustomerId, date, Quantity,
rank(Quantity) OVER (PARTITION BY CustomerId, date
ORDER BY Quantity DESC NULLS LAST
ROWS BETWEEN
UNBOUNDED PRECEDING AND
CURRENT ROW) as rank,

dense_rank(Quantity) OVER (PARTITION BY CustomerId, date
ORDER BY Quantity DESC NULLS LAST
ROWS BETWEEN
UNBOUNDED PRECEDING AND
CURRENT ROW) as dRank,

max(Quantity) OVER (PARTITION BY CustomerId, date
ORDER BY Quantity DESC NULLS LAST
ROWS BETWEEN
UNBOUNDED PRECEDING AND
CURRENT ROW) as maxPurchase
FROM df_with_date WHERE CustomerId IS NOT NULL ORDER BY CustomerId

CustomerId,date,Quantity,rank,dRank,maxPurchase
12346.0,2011-01-18,74215,1,1,74215
12346.0,2011-01-18,-74215,2,2,74215
12347.0,2010-12-07,30,2,2,36
12347.0,2011-10-31,12,14,6,48
12347.0,2011-08-02,24,2,2,36
12347.0,2011-10-31,20,11,4,48
12347.0,2011-10-31,36,2,2,48
12347.0,2011-10-31,12,14,6,48
12347.0,2010-12-07,6,17,5,36
12347.0,2011-08-02,12,8,4,36
